In [4]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
from cu2o_bulk import cu2o_bulk, cu2o111, CuD_FCC111, STO_FCC111, CuDO_FCC111

bulk = cu2o_bulk()
bulk.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')
ucf = UnitCellFilter(bulk)
LBFGS(ucf).run(fmax=0.01)
E_bulk = bulk.get_potential_energy()
E_bulk = bulk.get_potential_energy()

n_layers=3
slab = CuDO_FCC111(bulk, 3, 10)
slab.calc = mace_mp(model="large", dispersion=True, default_dtype="float64", device='cuda')

E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')


bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))

qn = LBFGS(slab, trajectory='CuDO111.traj')
write('CuDO111.traj', slab)
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
#t = read('111slab.traj@:')
#atoms = t[-1]
#view(atoms)
  
E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}')

'\nE_slab = slab.get_potential_energy()\nE_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f\'{n_layers=} {E_cleav=}\')\n\nslab = CuDO_FCC111(n_layers=5, vacuum=10)\n#slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device=\'cpu\')\n\nbottom_Cu_z = np.min(slab[slab.symbols==\'Cu\'].positions[:,2])\nmask1=slab.positions[:, 2] < bottom_Cu_z + 1.0\nslab.set_constraint(FixAtoms(mask=mask1))\n\n#qn = LBFGS(slab, trajectory=\'111slab.traj\')\n#qn.run(fmax=0.01)\n#E_slab = slab.get_potential_energy()\nt = read(\'111slab.traj@:\')\natoms = t[-1]\nview(atoms)\n  \nE_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f\'{n_layers=} {E_surf=}\')\n'